We run the demo from the README of `descwl-shear-sims`:

In [ ]:
import numpy as np

# Galaxies with fixed size and flux
from descwl_shear_sims.galaxies import FixedGalaxyCatalog

from descwl_shear_sims.sim import make_sim

# convenience function to make a PSF
from descwl_shear_sims.psfs import make_fixed_psf

seed = 8312
rng = np.random.RandomState(seed)

ntrial = 2
coadd_dim = 351
buff = 50

for trial in range(ntrial):
    print('trial: %d/%d' % (trial+1, ntrial))

    # galaxy catalog; you can make your own
    galaxy_catalog = FixedGalaxyCatalog(
        rng=rng,
        coadd_dim=coadd_dim,
        buff=buff,
        layout='random',
        mag=25,
        hlr=1.0,
    )

    # make a constant gaussian psf
    psf = make_fixed_psf(psf_type='gauss')

    # generate some simulation data, with a particular shear

    sim_data = make_sim(
        rng=rng,
        galaxy_catalog=galaxy_catalog,
        coadd_dim=coadd_dim,
        g1=0.02,
        g2=0.00,
        psf=psf,
    )

    # get the first i band exposure
    sim_data['band_data']['i'][0]

    # full list of outputs in the dictionary
    # sim_data: dict
    #     band_data: a dict keyed by band name, holding a list of Exposure
    #        objects
    #     coadd_wcs: lsst.afw.geom.makeSkyWcs
    #     psf_dims: (int, int)
    #     coadd_dims: (int, int)
    #     coadd_bbox: lsst.geom.Box2I
    #     bright_info: structured array
    #         fields are
    #         ra, dec: sky position of bright stars
    #         radius_pixels: radius of mask in pixels
    #         has_bleed: bool, True if there is a bleed trail
    #     se_wcs: list of WCS